Displays unweighted plots from the corresponding Coffea outputs.  The plots are categorized according to the histogram name (dependant variable) and the tag category defined in the `TTbarResProcessor`.  
# NOTE: #
All QCD MC histograms are normalized directly to the data, as no corrections are applied via mistag analysis or modmass procedures anyways.  For a more realistic analysis, refer to `TTbarResCoffea_BkgEstAnalysis` notebook.

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import os
from coffea import hist
from coffea import util
import numpy as np
import itertools

In [ ]:
dir1 = 'CoffeaOutputs/UnweightedOutputs/'

QCD_unweighted = util.load(dir1+'TTbarResCoffea_QCD_unweighted_output.coffea')
TTbar_unweighted = util.load(dir1+'TTbarResCoffea_TTbar_unweighted_output.coffea')
JetHT2016_unweighted = util.load(dir1+'TTbarResCoffea_JetHT2016_Data_unweighted_output_futures_3-10-21_trial.coffea')
RSGluon1000_unweighted = util.load(dir1+'TTbarResCoffea_RSGluon1000_unweighted_output.coffea')

In [ ]:
Datasets = {
    'QCD': QCD_unweighted,
    'TTbar': TTbar_unweighted,
    'JetHT2016': JetHT2016_unweighted,
    'RSGluon1000': RSGluon1000_unweighted
}

# The cutflow can be checked if desired 

In [ ]:
for dataset,output in Datasets.items():
    print("-------" + dataset + " Cutflow--------")
    for i,j in output['cutflow'].items():        
        print( '%20s : %12d' % (i,j) )

In [ ]:
def mkdir_p(mypath):
    '''Creates a directory. equivalent to using mkdir -p on the command line'''

    from errno import EEXIST
    from os import makedirs,path

    try:
        makedirs(mypath)
    except OSError as exc: # Python >2.5
        if exc.errno == EEXIST and path.isdir(mypath):
            pass
        else: raise

In [ ]:
def DoesDirectoryExist(mypath): #extra precaution (Probably overkill...)
    '''Checks to see if Directory exists before running mkdir_p'''
    import os.path
    from os import path
    
    if path.exists(mypath):
        pass
    else:
        mkdir_p(mypath)

In [ ]:
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import warnings
import re # regular expressions
warnings.filterwarnings("ignore")

# ---- Reiterate categories ---- #
ttagcats = ["Probet", "at", "0t", "pret", "1t", "1t+2t", "2t", "0t+1t+2t"] # 'Pt' = t-tagged probe
btagcats = ["0b", "1b", "2b"]
ycats = ['cen', 'fwd']

list_of_cats = [ t+b+y for t,b,y in itertools.product( ttagcats, btagcats, ycats) ]

# ---- List the Histograms Here ---- #
list_of_hists_4vector = ('SDmass', 'tau32', 'jetmass', 'ttbarmass', 'jetpt', 'jeteta', 'jetphi', 'jety', 'jetdy')

In [ ]:
maindirectory = os.getcwd() 

In [ ]:
""" ---------------- Luminosity and Cross Sections ---------------- """
Lum     = 137190. # pb^-1 from https://twiki.cern.ch/twiki/bin/viewauth/CMS/PdmVAnalysisSummaryTable
Lum2016 = 35920.

tt_mc_entire_sumw = 858189552.
ttbar_BR = 0.45 #Calculated using PDG 2019
ttbar_xs = 1.0 #831.76*ttbar_BR  #pb Monte Carlo already includes xs in event weight!!
ttbar_sf2016 = ttbar_xs*Lum2016/142155064.
print(ttbar_sf2016)

qcd_xs = 1370000000.0 #pb From https://cms-gen-dev.cern.ch/xsdb

In [ ]:
Nevts2016 = 625516390. # from dasgoclient
Nevts2017 = 410461585.
Nevts2018 = 676328827.
Nevts = Nevts2016 + Nevts2017 + Nevts2018
Nevts_sf2016 = Nevts2016/JetHT2016_unweighted['cutflow']['all events']
print(Nevts_sf2016)

In [ ]:
stack_tag1_opts = {'marker': '.', 'markersize': 10., 'color': 'b', 'elinewidth': 1}
stack_tag2_opts = {'marker': 's', 'markersize': 5., 'color': 'g', 'elinewidth': 1}
legend_tag_opts = {'labels':['', '', 'DeepTag', 'DeepTag MD']}
stack_ttbar_opts = {'alpha': 0.8, 'edgecolor':(0,0,0,0.3), 'color': 'red'}
stack_background_opts = {'alpha': 0.8, 'edgecolor':(0,0,0,0.3), 'color': 'yellow'}
stack_error_opts = {'label':'Stat. Unc.', 'hatch':'///', 'facecolor':'None', 'edgecolor':(0,0,0,.5), 'linewidth': 0}
data_err_opts = {'linestyle': 'none', 'marker': '.', 'markersize': 10., 'color': 'k', 'elinewidth': 1}

# Data and Unweighted MC Plots
### NOTE that SDmass used axes called jetmass, so code will get confused unless exception is made for SDmass

In [ ]:
# for dataset,output in Datasets.items():
for ihist in list_of_hists_4vector:
    SaveDirectory = maindirectory + '/' + ihist + '/' # split histograms into subdirectories
    DoesDirectoryExist(SaveDirectory) # no need to create the directory several times if it exists already
    for icat in list_of_cats: 
        plt.rcParams.update({
        'font.size': 14,
        'axes.titlesize': 18,
        'axes.labelsize': 18,
        'xtick.labelsize': 12,
        'ytick.labelsize': 12
        })
        fig, (ax, rax) = plt.subplots(
            nrows=2,
            ncols=1,
            figsize=(7,7),
            gridspec_kw={"height_ratios": (3, 1)},
            sharex=True
        )
        fig.subplots_adjust(hspace=.07)
        title = ihist + '  ' + icat
        #filename = ihist + '_' + icat + '_LinearScale.' + 'png'
        filename = ihist + '_' + icat + '.' + 'png'
        legend_labels = {'labels':['', '', '', '', 'QCD Bkg (Sim.)', r'$t\bar{t}$ Sim.', 'Data'],
                         'loc': 'upper right',
                         'fontsize': 'xx-small'}
        
        #---- Define Scaled Histograms by category and scale the Total----# 

#    ===================================================================================================================
#    N     N   OOO   RRRRRR  M     M    A    L       IIIIIII ZZZZZZZ EEEEEEE     TTTTTTT TTTTTTT BBBBBB     A    RRRRRR      
#    NN    N  O   O  R     R MM   MM   A A   L          I         Z  E              T       T    B     B   A A   R     R     
#    N N   N O     O R     R M M M M  A   A  L          I        Z   E              T       T    B     B  A   A  R     R     
#    N  N  N O     O RRRRRR  M  M  M  AAAAA  L          I       Z    EEEEEEE        T       T    BBBBBB   AAAAA  RRRRRR      
#    N   N N O     O R   R   M     M A     A L          I      Z     E              T       T    B     B A     A R   R       
#    N    NN  O   O  R    R  M     M A     A L          I     Z      E              T       T    B     B A     A R    R      
#    N     N   OOO   R     R M     M A     A LLLLLLL IIIIIII ZZZZZZZ EEEEEEE        T       T    BBBBBB  A     A R     R 
#    ===================================================================================================================
        
        # ---- A copy of the ttbar hist will be made and added to QCD hist to simulate the SM background ---- #
        TTbar_hist = TTbar_unweighted[ihist].integrate('anacat', icat).integrate('dataset', 'TTbar')
        MC_hist = TTbar_hist.copy() # Used for SM background
        if ihist == 'SDmass':
            Ntt = np.sum(MC_hist.integrate('jetmass').values()) # extract number of events from histogram directly
        else:
            Ntt = np.sum(MC_hist.integrate(ihist).values())
        Ntt = [i for i in Ntt.values()] # the zeroth column entry is the number of events defined as a float
        if Ntt[0] > 0.:
            MC_hist.scale(ttbar_xs/Ntt[0]) # Normalized to ttbar xs
        else:
            MC_hist.scale(0.)

#    ===================================================================================================
#    N     N   OOO   RRRRRR  M     M    A    L       IIIIIII ZZZZZZZ EEEEEEE       QQQ     CCCC  DDDD        
#    NN    N  O   O  R     R MM   MM   A A   L          I         Z  E            Q   Q   C      D   D       
#    N N   N O     O R     R M M M M  A   A  L          I        Z   E           Q     Q C       D    D      
#    N  N  N O     O RRRRRR  M  M  M  AAAAA  L          I       Z    EEEEEEE     Q     Q C       D     D     
#    N   N N O     O R   R   M     M A     A L          I      Z     E           Q   Q Q C       D    D      
#    N    NN  O   O  R    R  M     M A     A L          I     Z      E            Q   Q   C      D   D       
#    N     N   OOO   R     R M     M A     A LLLLLLL IIIIIII ZZZZZZZ EEEEEEE       QQQ Q   CCCC  DDDD
#    ===================================================================================================
        
        QCD_hist = QCD_unweighted[ihist].integrate('anacat', icat).integrate('dataset', 'QCD')
        if ihist == 'SDmass':
            Nqcd = np.sum(QCD_hist.integrate('jetmass').values())
        else:
            Nqcd = np.sum(QCD_hist.integrate(ihist).values())
        Nqcd = [j for j in Nqcd.values()]
        if Nqcd[0] > 0.:
            QCD_hist.scale(qcd_xs/Nqcd[0]) # Normalized to qcd xs
        else:
            QCD_hist.scale(0.)
        
        # ---- Add QCD cross section histogram to ttbar xs histogram ---- #
        MC_hist.add(QCD_hist) # Adds Normalized MC Histograms to get Total MC
        
#    ===================================================================================================================
#    DDDD       A    TTTTTTT    A        H     H IIIIIII   SSSSS TTTTTTT   OOO   GGGGGGG RRRRRR     A    M     M   SSSSS     
#    D   D     A A      T      A A       H     H    I     S         T     O   O  G       R     R   A A   MM   MM  S          
#    D    D   A   A     T     A   A      H     H    I    S          T    O     O G       R     R  A   A  M M M M S           
#    D     D  AAAAA     T     AAAAA      HHHHHHH    I     SSSSS     T    O     O G  GGGG RRRRRR   AAAAA  M  M  M  SSSSS      
#    D    D  A     A    T    A     A     H     H    I          S    T    O     O G     G R   R   A     A M     M       S     
#    D   D   A     A    T    A     A     H     H    I         S     T     O   O  G     G R    R  A     A M     M      S      
#    DDDD    A     A    T    A     A     H     H IIIIIII SSSSS      T      OOO    GGGGG  R     R A     A M     M SSSSS
#    ===================================================================================================================
        
        # ---- Define the data histogram ---- #
        Data_hist = JetHT2016_unweighted[ihist].integrate('anacat', icat).integrate('dataset', 'JetHT2016_Data')
        Data_hist.scale(Nevts_sf2016) #scale according to number of events in dataset
        
        # ---- Extract both the data and MC events and from histograms ---- #
        if ihist == 'SDmass':
            NtotalMC = np.sum(MC_hist.integrate('jetmass').values())
            NtotalData = np.sum(Data_hist.integrate('jetmass').values())
        else:
            NtotalMC = np.sum(MC_hist.integrate(ihist).values())
            NtotalData = np.sum(Data_hist.integrate(ihist).values())
        NtotalMC = [k for k in NtotalMC.values()]
        NtotalData = [l for l in NtotalData.values()]
        
        # ---- Normalize the total MC histogram directly to the data (for aesthetic purposes only!) ---- #
        # ---- Unweighted simulation of the background alone does not accurately fit the data ---- #
        if NtotalMC[0] > 0.:
            MC_hist.scale(NtotalData[0]/NtotalMC[0])
        else:
            MC_hist.scale(0.)
            
#    ===========================================================================
#    M     M    A    K     K EEEEEEE     PPPPPP  L         OOO   TTTTTTT   SSSSS     
#    MM   MM   A A   K   K   E           P     P L        O   O     T     S          
#    M M M M  A   A  K K     E           P     P L       O     O    T    S           
#    M  M  M  AAAAA  KKk     EEEEEEE     PPPPPP  L       O     O    T     SSSSS      
#    M     M A     A K  K    E           P       L       O     O    T          S     
#    M     M A     A K   K   E           P       L        O   O     T         S      
#    M     M A     A K   K   EEEEEEE     P       LLLLLLL   OOO      T    SSSSS 
#    ===========================================================================
        
        #---- Plot Total MC (simulated QCD + SM ttbar background)----#
        #-----------------------------------------------------------------#
        hist.plot1d(MC_hist, ax=ax, clear=False,
                    fill_opts=stack_background_opts, error_opts=stack_error_opts)
        
        #---- Plot TTbar MC for comparison ---- #
        #-----------------------------------------------------------------#
        if Ntt[0] > 0:
            TTbar_hist.scale(ttbar_sf2016) # Scale to Luminosity and adjust for MC generator weight
            #TTbar_hist.scale(NtotalData[0]/Ntt[0]) # Scale up to data
            #TTbar_hist.scale(35922.*0.82*831.76/(77229341 + 78006311)) #From original analysis
        else:
            TTbar_hist.scale(0.)
            
        hist.plot1d(TTbar_hist, ax=ax, clear=False,
                    fill_opts=stack_ttbar_opts, error_opts=stack_error_opts)
        
        #---- Plot Data ----#
        #-----------------------------------------------------------------#
        hist.plot1d(Data_hist, ax=ax, clear=False, 
                    error_opts=data_err_opts,
                    legend_opts=legend_labels)
        
        #---- Plot RSGluon ----#
        #-----------------------------------------------------------------#
#         .
#         .
#         .
        
        ax.set_ylim(bottom=0.1)
        ax.set_yscale('log')
        ax.set_ylabel('Events')
        ax.set_xlabel(None)
        ax.set_title(title)
        #leg = ax.legend(labels=["QCD Sim. Bkg", r'$t\bar{t}$ Sim.', 'Data'])
        
        
        #---- Plot Ratio ----#
        hist.plotratio(num = Data_hist, denom = MC_hist, ax = rax,
                       error_opts={'marker': '.', 'markersize': 10., 'color': 'k', 'elinewidth': 1},
                       unc = 'num')
        rax.set_ylabel('Data/MC')
        rax.axhline(y=1, color='k', linestyle=':')
        rax.set_ylim(0,2)
        
        if ihist == 'ttbarmass':
            rax.set_xlim(700,5000)
        if ihist == 'jetpt':
            rax.set_xlim(0,3000)
        if ihist == 'jeteta':
            rax.set_xlim(-3,3)
        if ihist == 'tau32':
            rax.set_xlim(0,1.2)
       
        #---- Labeling ----#
        Lint = str(Lum2016*.001) # Integrated Luminosity
        lumi = plt.text(1.15, 1.07, Lint[:6] + " fb$^{-1}$",
                fontsize=16,
                horizontalalignment='right',
                verticalalignment='top',
                transform=ax.transAxes
               )
        #plt.savefig(SaveDirectory+filename, bbox_inches="tight")
        print(filename + ' saved')